<a href="https://colab.research.google.com/github/slazur83/Tableau/blob/main/import_steps_from_garmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install garminconnect

In [16]:
from garminconnect import Garmin
import pandas as pd
from datetime import date, timedelta
import os

# Paths
TOKEN_DIR = "/content/drive/MyDrive/Programowanie/Google Colab/Tableau/Sport_Activity/Garmin tokens"
BASE_DIR = "/content/drive/MyDrive/Programowanie/Google Colab/Tableau/Sport_Activity"
IMPORT_DIR = os.path.join(BASE_DIR, "Garmin imports")

os.makedirs(IMPORT_DIR, exist_ok=True)

# Login
garmin = Garmin()
garmin.login(TOKEN_DIR)

# Verify login
try:
    garmin.get_user_profile()
except Exception as e:
    raise Exception(f"Login failed! Check your tokens. Details: {e}")

# Choose date range option
print("Choose start date option:")
print("1: From Garmin account start (21 Dec 2020)")
print("2: From beginning of current year")
print("3: Enter custom start date")
choice = input("Enter 1, 2, or 3: ").strip()

if choice == "1":
    start = date(2020, 12, 21)
elif choice == "2":
    start = date(date.today().year, 1, 1)
elif choice == "3":
    user_input = input("Enter custom start date (YYYY-MM-DD): ").strip()
    try:
        start = date.fromisoformat(user_input)
    except ValueError:
        print("Invalid format. Using start of current year.")
        start = date(date.today().year, 1, 1)
else:
    print("Invalid choice. Using start of current year.")
    start = date(date.today().year, 1, 1)

end = date.today()

# Fetch data
rows = []
current = start
while current <= end:
    iso = current.isoformat()
    try:
        summary = garmin.get_user_summary(iso)
        steps = summary.get("totalSteps") or summary.get("steps") or 0
    except Exception as e:
        print(f"Error for {iso}: {e}")
        steps = 0

    rows.append({"date": iso, "steps": steps})
    current += timedelta(days=1)

df = pd.DataFrame(rows)

# Create file name with date range
file_name = f"steps_{start.isoformat()}_to_{end.isoformat()}.csv"
file_path = os.path.join(IMPORT_DIR, file_name)

df.to_csv(file_path, index=False)
print(f"File saved to {file_path}")


Choose start date option:
1: From Garmin account start (21 Dec 2020)
2: From beginning of current year
3: Enter custom start date
Enter 1, 2, or 3: 2
File saved to /content/drive/MyDrive/Programowanie/Google Colab/Tableau/Sport_Activity/Garmin imports/steps_2025-01-01_to_2025-11-30.csv
